# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>A z-test comparing the proportions of the two samples  is appropriate.
The saple size is large enough at 4870, and less than 10% of individuals seeking jobs to assume independence and compliance with the CLT.

H<sub>o</sub> = P(white applicant call back rate) = P(black applicant call back rate)

H<sub>a</sub> = P(white applicant call back rate) > P(black applicant call back rate)</p>
</div>

In [11]:
w = data[data.race=='w']
b = data[data.race=='b']

In [20]:
wcalls = sum(w.call)
wapps = len(w)

bcalls = sum(b.call)
bapps = len(b)

w_call_rate = wcalls / wapps

b_call_rate = bcalls / bapps

total_rate = (wcalls + bcalls) / (wapps + bapps)

print('number of white call backs:', wcalls, 'number of white applicants:', wapps)
print('number of black call backs:', bcalls, 'number of black applicants:', bapps)
print('the call back rate for white applicants is', w_call_rate, \
     'the call back rate for black applicants is', b_call_rate)
print('the call back rate for all applicants is:', total_rate)

number of white call backs: 235.0 number of white applicants: 2435
number of black call backs: 157.0 number of blacke applicants: 2435
the call back rate for white applicants is 0.09650924024640657 the call back rate for black applicants is 0.06447638603696099
the call back rate for all applicants is: 0.08049281314168377


In [55]:
# Your solution to Q3 here

# Frequentist Method
empirical_diff_means = (w_call_rate - b_call_rate)
standard_error_means = np.sqrt(((total_rate * (1-total_rate)) / wapps) + ((total_rate * (1-total_rate)) / bapps))
z_norm = stats.norm.ppf(.975) #95% confidence interval
MoE = z_norm * standard_error_means
print('empirical difference of means:', empirical_diff_means)
print('standard error / standard deviation of difference of means:', standard_error_means)
print('margin of error at 95% confidence interval:', MoE)

z_test = empirical_diff_means / standard_error_means
p_value = 1 - stats.norm.cdf(z_test)
print('Start with an alpha = 0.05. With a Z test statistic of', z_test, 'which calcultes to a p-value of', p_value, \
     'that is well below our alpha of 0.05, we reject the null hypothesis that the white applicant call back rate ' \
      'is equal the black applicant call back rate.')

# create data functions
WCall = w.call
BCall = b.call


# create difference of means function
def diff_of_means(df1, df2):
    
    delta = np.mean(df1) - np.mean(df2)
    
    return delta

# create bootstrap replication function
def draw_bs_reps(df, sampsize, func, size=1):
    """Get bootstrap replicate[s] of a sample"""
    
    bs_replicates = np.empty(size)
    
    for i in range(size):        
        bs_replicates[i] = func(np.random.choice(df, sampsize))
    
    return bs_replicates

bs_reps_w = draw_bs_reps(WCall, len(WCall), np.mean, 10000)
bs_reps_b = draw_bs_reps(BCall, len(BCall), np.mean, 10000)
bs_reps_diff = bs_reps_w - bs_reps_b
MoE_bs = bs_reps_diff.std() * stats.norm.ppf(.975)
bs_conf_int = np.percentile(bs_reps_diff, [2.5,97.5])

print("margin of error - bootstrap replicates:", MoE_bs)
print("95% confidence interval - bootstrap replicates:", bs_conf_int)


# Set up permutation sample functions
def permutation_sample(df1, df2):
    
    data = np.concatenate((df1, df2)) #concatenate data sets
    
    permuted_data = np.random.permutation(data)
    
    #split permuted data
    perm_sample_1 = permuted_data[:len(df1)]
    perm_sample_2 = permuted_data[len(df1):]
    
    return perm_sample_1, perm_sample_2

def draw_perm_reps(df1, df2, func, size=1):
    
    perm_replicates = np.empty(size)
    
    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(df1, df2)
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    
    return perm_replicates

perm_reps = draw_perm_reps(WCall, BCall, diff_of_means, 10000)
p_perm_reps = np.sum(perm_reps >= empirical_diff_means) / len(perm_reps)
print('p-value from permutaion replications:', p_perm_reps)



empirical difference of means: 0.032032854209445585
standard error / standard deviation of difference of means: 0.007796894036170457
margin of error at 95% confidence interval: 0.015281631502169232
Start with an alpha = 0.05. With a Z test statistic of 4.108412152434346 which calcultes to a p-value of 1.991943418788722e-05 that is well below our alpha of 0.05, we reject the null hypothesis that the white applicant call back rate is equal the black applicant call back rate.
margin of error - bootstrap replicates: 0.0151970205941569
95% confidence interval - bootstrap replicates: [0.01683778 0.0476386 ]
p-value from permutaion replications: 0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
In this experiment, of 4,870 resumes, half were randomly titled with "black sounding" names and half were titled with "white sounding" names to determine if there would be a difference in the rate at which the applicants were called back for interviews.  The data from this experiment shows the rate of call backs for white applicants is 3.2 percentage points higher than for black applicants.  With a margin of error at approximately 1.5 percentage points, this difference is clearly outside the margin of error.  This shows race, or perceived race, is clearly a factor in the hiring process.

While clearly an important factor,  it cannot be determined if it is the most important factor.  In order to know this, we would have to undergo a similar process to test for factors such as education, industry applied to, etc.</div>
